In [1]:
%load_ext autoreload
%autoreload 2
import pandas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import pylab

from docx import Document
from docx.shared import Pt
import traceback
from docx.oxml.shared import OxmlElement, qn

from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
import seaborn as sns
pandas.set_option('display.float_format', lambda x: '%.1f' % x)
%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
from ggb_constants import GGB_SECTORS 

## Parse input

In [3]:
df = pandas.read_csv("exo_one/exo_one.csv")
df.head()
kad = df["ΚΑΔ Επένδυσης"].str.split(".", -1, True).apply(pd.to_numeric, errors="ignore")
df = df.assign(
    d2=kad[0],
    d4=kad[1],
    d6=kad[2],
    d8=kad[3],
    size=df.ΠΕΠ.str.rsplit("(", 1, True)[1].str[:-1],
)
df.head()
df = df.drop(['ΠΕΠ', 'Κωδικός Πρότασης', 'Εταίρος ΕΦΔ', 'Επωνυμία Επιχείρησης', 'Περιγραφή ΚΑΔ Επένδυσης'], axis=1)
df.head()
df = df.rename(columns={
    "ΚΑΔ Επένδυσης": "kad",
    "Περιφέρεια Υλοποίησης": "prefecture",
    "Συνολικός Προϋπολογισμός": "budget",
    "Επιχορηγούμενος Προϋπολογισμός": "subsidized",
    "Δημόσια Δαπάνη Ολοκλήρωσης": "granted",
})
df.head()
df = df.assign(sector=df.d2.map(GGB_SECTORS))
df.head()

,Εταίρος ΕΦΔ,Κωδικός Πρότασης,Επωνυμία Επιχείρησης,ΚΑΔ Επένδυσης,Περιγραφή ΚΑΔ Επένδυσης,Περιφέρεια Υλοποίησης,ΠΕΠ,Συνολικός Προϋπολογισμός,Επιχορηγούμενος Προϋπολογισμός,Δημόσια Δαπάνη Ολοκλήρωσης
0,ΑΕΔΕΠ,ΕΞ-00353,ΣΠΥΡΟΥ Δ. ΚΑΙ ΣΙΑ Ο.Ε.,23.70.12,"Κοπή, μορφοποίηση και τελική επεξεργασία άλλων...",ΘΕΣΣΑΛΙΑ,ΕΠ Ανταγωνιστικότητα και Επιχειρηματικότητα - ...,50748.1,47450.0,21352.5
1,ΑΕΔΕΠ,ΕΞ-00744,XAΛΑΤΣΗΣ ΕΥΑΓΓΕΛΟΣ,27.12.31,"Κατασκευή πινάκων και άλλων διατάξεων, εξοπλισ...",ΘΕΣΣΑΛΙΑ,ΕΠ Ανταγωνιστικότητα και Επιχειρηματικότητα - ...,153000.0,153000.0,68850.0
2,ΑΕΔΕΠ,ΕΞ-00894,ΓΕΩΡΓΙΟΣ ΚΑΤΣΙΦΑΣ ΑΝΩΝΥΜΗ ΒΙΟΤΕΧΝΙΚΗ ΚΑΙ ΕΜΠΟΡ...,10.84.2,"Παραγωγή μπαχαρικών, που διαθέτονται κατεργασμένα",ΘΕΣΣΑΛΙΑ,ΕΠ Ανταγωνιστικότητα και Επιχειρηματικότητα - ...,42188.0,42188.0,18984.6
3,ΑΕΔΕΠ,ΕΞ-01016,ΒΙΟΜΗΧΑΝΙΑ ΠΛΑΣΤΙΚΩΝ ΛΑΡΙΠΛΑΣΤ ΑΝΩΝΥΜΗ ΕΜΠΟΡΙΚ...,20.16,Παραγωγή πλαστικών σε πρωτογενείς μορφές,ΘΕΣΣΑΛΙΑ,ΕΠ Ανταγωνιστικότητα και Επιχειρηματικότητα - ...,70820.0,70820.0,31869.0
4,ΑΕΔΕΠ,ΕΞ-01180,ΜΠΟΜΠΟΛΑΣ ΑΠΟΣΤΟΛΟΣ Α.Ε. ΣΥΣΤΗΜΑΤΩΝ ΦΩΤΟΒΟΛΤΑΙ...,42.22.23,Κατασκευαστικές εργασίες για μονάδες παραγωγής...,ΘΕΣΣΑΛΙΑ,ΕΠ Ανταγωνιστικότητα και Επιχειρηματικότητα - ...,92048.4,92048.4,41421.8


,Εταίρος ΕΦΔ,Κωδικός Πρότασης,Επωνυμία Επιχείρησης,ΚΑΔ Επένδυσης,Περιγραφή ΚΑΔ Επένδυσης,Περιφέρεια Υλοποίησης,ΠΕΠ,Συνολικός Προϋπολογισμός,Επιχορηγούμενος Προϋπολογισμός,Δημόσια Δαπάνη Ολοκλήρωσης,d2,d4,d6,d8,size
0,ΑΕΔΕΠ,ΕΞ-00353,ΣΠΥΡΟΥ Δ. ΚΑΙ ΣΙΑ Ο.Ε.,23.70.12,"Κοπή, μορφοποίηση και τελική επεξεργασία άλλων...",ΘΕΣΣΑΛΙΑ,ΕΠ Ανταγωνιστικότητα και Επιχειρηματικότητα - ...,50748.1,47450.0,21352.5,23,70,12.0,nan,Μικρές και Πολύ Μικρές
1,ΑΕΔΕΠ,ΕΞ-00744,XAΛΑΤΣΗΣ ΕΥΑΓΓΕΛΟΣ,27.12.31,"Κατασκευή πινάκων και άλλων διατάξεων, εξοπλισ...",ΘΕΣΣΑΛΙΑ,ΕΠ Ανταγωνιστικότητα και Επιχειρηματικότητα - ...,153000.0,153000.0,68850.0,27,12,31.0,nan,Μικρές και Πολύ Μικρές
2,ΑΕΔΕΠ,ΕΞ-00894,ΓΕΩΡΓΙΟΣ ΚΑΤΣΙΦΑΣ ΑΝΩΝΥΜΗ ΒΙΟΤΕΧΝΙΚΗ ΚΑΙ ΕΜΠΟΡ...,10.84.2,"Παραγωγή μπαχαρικών, που διαθέτονται κατεργασμένα",ΘΕΣΣΑΛΙΑ,ΕΠ Ανταγωνιστικότητα και Επιχειρηματικότητα - ...,42188.0,42188.0,18984.6,10,84,2.0,nan,Μικρές και Πολύ Μικρές
3,ΑΕΔΕΠ,ΕΞ-01016,ΒΙΟΜΗΧΑΝΙΑ ΠΛΑΣΤΙΚΩΝ ΛΑΡΙΠΛΑΣΤ ΑΝΩΝΥΜΗ ΕΜΠΟΡΙΚ...,20.16,Παραγωγή πλαστικών σε πρωτογενείς μορφές,ΘΕΣΣΑΛΙΑ,ΕΠ Ανταγωνιστικότητα και Επιχειρηματικότητα - ...,70820.0,70820.0,31869.0,20,16,nan,nan,Μεσαίες
4,ΑΕΔΕΠ,ΕΞ-01180,ΜΠΟΜΠΟΛΑΣ ΑΠΟΣΤΟΛΟΣ Α.Ε. ΣΥΣΤΗΜΑΤΩΝ ΦΩΤΟΒΟΛΤΑΙ...,42.22.23,Κατασκευαστικές εργασίες για μονάδες παραγωγής...,ΘΕΣΣΑΛΙΑ,ΕΠ Ανταγωνιστικότητα και Επιχειρηματικότητα - ...,92048.4,92048.4,41421.8,42,22,23.0,nan,Μικρές και Πολύ Μικρές


,ΚΑΔ Επένδυσης,Περιφέρεια Υλοποίησης,Συνολικός Προϋπολογισμός,Επιχορηγούμενος Προϋπολογισμός,Δημόσια Δαπάνη Ολοκλήρωσης,d2,d4,d6,d8,size
0,23.70.12,ΘΕΣΣΑΛΙΑ,50748.1,47450.0,21352.5,23,70,12.0,nan,Μικρές και Πολύ Μικρές
1,27.12.31,ΘΕΣΣΑΛΙΑ,153000.0,153000.0,68850.0,27,12,31.0,nan,Μικρές και Πολύ Μικρές
2,10.84.2,ΘΕΣΣΑΛΙΑ,42188.0,42188.0,18984.6,10,84,2.0,nan,Μικρές και Πολύ Μικρές
3,20.16,ΘΕΣΣΑΛΙΑ,70820.0,70820.0,31869.0,20,16,nan,nan,Μεσαίες
4,42.22.23,ΘΕΣΣΑΛΙΑ,92048.4,92048.4,41421.8,42,22,23.0,nan,Μικρές και Πολύ Μικρές


,kad,prefecture,budget,subsidized,granted,d2,d4,d6,d8,size
0,23.70.12,ΘΕΣΣΑΛΙΑ,50748.1,47450.0,21352.5,23,70,12.0,nan,Μικρές και Πολύ Μικρές
1,27.12.31,ΘΕΣΣΑΛΙΑ,153000.0,153000.0,68850.0,27,12,31.0,nan,Μικρές και Πολύ Μικρές
2,10.84.2,ΘΕΣΣΑΛΙΑ,42188.0,42188.0,18984.6,10,84,2.0,nan,Μικρές και Πολύ Μικρές
3,20.16,ΘΕΣΣΑΛΙΑ,70820.0,70820.0,31869.0,20,16,nan,nan,Μεσαίες
4,42.22.23,ΘΕΣΣΑΛΙΑ,92048.4,92048.4,41421.8,42,22,23.0,nan,Μικρές και Πολύ Μικρές


,kad,prefecture,budget,subsidized,granted,d2,d4,d6,d8,size,sector
0,23.70.12,ΘΕΣΣΑΛΙΑ,50748.1,47450.0,21352.5,23,70,12.0,nan,Μικρές και Πολύ Μικρές,Μεταποίηση
1,27.12.31,ΘΕΣΣΑΛΙΑ,153000.0,153000.0,68850.0,27,12,31.0,nan,Μικρές και Πολύ Μικρές,Μεταποίηση
2,10.84.2,ΘΕΣΣΑΛΙΑ,42188.0,42188.0,18984.6,10,84,2.0,nan,Μικρές και Πολύ Μικρές,Μεταποίηση
3,20.16,ΘΕΣΣΑΛΙΑ,70820.0,70820.0,31869.0,20,16,nan,nan,Μεσαίες,Μεταποίηση
4,42.22.23,ΘΕΣΣΑΛΙΑ,92048.4,92048.4,41421.8,42,22,23.0,nan,Μικρές και Πολύ Μικρές,Κατασκευές


## Function definitions

Just some basic building blocks

In [30]:
def get_grouped_data(df, col, by):
    group = df[[col, by]].groupby([by])
    absolute = group.sum()
    out = pd.DataFrame(index=absolute.index)
    out = out.assign(
        absolute=absolute,
        relative=(absolute / absolute.sum() * 100),
        average=group.mean(),
    )
    return out

def get_no_projects(df, by):
    abs_no_proj = df.groupby(by).size().rename('N')
    rel_no_proj = (abs_no_proj / abs_no_proj.sum() * 100).rename('p')
    return pd.concat([abs_no_proj, rel_no_proj], axis=1)

def concat_groups(budget, subsidized, granted, column):
    return pd.concat(
        [
            budget[column].rename("budget"),
            subsidized[column].rename("subsidized"),
            granted[column].rename("granted"),
        ],
        axis=1
    )

And now the functions that actually build the tables    

### Format A

This format IMHO makes the table the most readable, but you need to add a legend-line to each table. E.g.

> Υπόμνημα: ΣΠ -> Συνολικός Προϋπολογισμός, ΕΠ -> Επιχορηγούμενος Προϋπολογισμός, ΔΔΟ -> Δημόσια Δαπάνης Ολοκλήρωσης

In [31]:
RENAMES1 = {
    "projects": "Έργα",
    "budget": "ΣΠ",
    "subsidized": "ΕΠ",
    "granted": "ΔΔΠ",
    "absolute": "Kόστος (€x10^6)",
    "relative": "Κατανομή κόστους(%)",
    "average": "Mέσος όρος (€x10^6)",
    "N": "Ν",
    "p": "(%)",
    "size": "Μέγεθος",
    "sector": "Τομέας",
    "prefecture": "Περιφέρεια",
}

def create_grouped_table1(df, by, rename=True):
    budget = get_grouped_data(df, 'budget', by)
    granted = get_grouped_data(df, 'granted', by)
    subsidized = get_grouped_data(df, 'subsidized', by)

    table = pd.concat(
        [
            get_no_projects(df, by),
            concat_groups(budget, granted, subsidized, 'absolute'),
            concat_groups(budget, granted, subsidized, 'relative'),
            concat_groups(budget, granted, subsidized, 'average'),
        ],
        axis=1,
        keys=['projects', 'absolute', 'relative', 'average'],
    )
    if rename:
        table = table.reset_index().rename(columns=RENAMES1).set_index(RENAMES1[by])
    return table

create_grouped_table1(df, "size")
create_grouped_table1(df, "sector", )
create_grouped_table1(df, "prefecture")

Έργα      Kόστος (€x10^6)                        \
                          Ν  (%)              ΣΠ         ΕΠ        ΔΔΠ   
Μέγεθος                                                                  
Μεγάλες                   7  1.4       1343610.9   532853.9  1332696.5   
Μεσαίες                 117 24.1      16394101.5  7286229.2 16075613.0   
Μικρές και Πολύ Μικρές  362 74.5      37534707.8 16732041.8 36644001.4   

                       Κατανομή κόστους(%)           Mέσος όρος (€x10^6)  \
                                        ΣΠ   ΕΠ  ΔΔΠ                  ΣΠ   
Μέγεθος                                                                    
Μεγάλες                                2.4  2.2  2.5            191944.4   
Μεσαίες                               29.7 29.7 29.7            140120.5   
Μικρές και Πολύ Μικρές                67.9 68.2 67.8            103687.0   

                                         
                            ΕΠ      ΔΔΠ  
Μέγεθος                                  
Μεγάλες                76122.0 190385.2  
Μεσαίες                62275.5 137398.4  
Μικρές και Πολύ Μικρές 46221.1 101226.5

Έργα      Kόστος (€x10^6)                        \
                   Ν  (%)              ΣΠ         ΕΠ        ΔΔΠ   
Τομέας                                                            
Εμπόριο           23  4.7       3804814.5  1861148.9  3770496.4   
Επαγ/Επιστ. Δρ.   31  6.4       2130415.1   955797.9  2088316.7   
Κατασκευές        13  2.7        925794.4   404262.1   917316.7   
Μεταποίηση       315 64.8      37503875.9 16555866.1 36701685.0   
ΤΠΕ              102 21.0      10693094.3  4677934.5 10360070.2   
Υγεία              2  0.4        214426.0    96115.4   214426.0   

                Κατανομή κόστους(%)           Mέσος όρος (€x10^6)          \
                                 ΣΠ   ΕΠ  ΔΔΠ                  ΣΠ      ΕΠ   
Τομέας                                                                      
Εμπόριο                         6.9  7.6  7.0            165426.7 80919.5   
Επαγ/Επιστ. Δρ.                 3.9  3.9  3.9             68723.1 30832.2   
Κατασκευές                      1.7  1.6  1.7             71215.0 31097.1   
Μεταποίηση                     67.9 67.4 67.9            119059.9 52558.3   
ΤΠΕ                            19.3 19.1 19.2            104834.3 45862.1   
Υγεία                           0.4  0.4  0.4            107213.0 48057.7   

                          
                     ΔΔΠ  
Τομέας                    
Εμπόριο         163934.6  
Επαγ/Επιστ. Δρ.  67365.1  
Κατασκευές       70562.8  
Μεταποίηση      116513.3  
ΤΠΕ             101569.3  
Υγεία           107213.0

Έργα      Kόστος (€x10^6)                        \
                               Ν  (%)              ΣΠ         ΕΠ        ΔΔΠ   
Περιφέρεια                                                                    
ΑΝΑΤΟΛΙΚΗ ΜΑΚΕΔΟΝΙΑ & ΘΡΑΚΗ   15  3.1       1582200.9   698685.9  1534724.0   
ΑΤΤΙΚΗ                       244 50.2      25310522.2 11186104.6 24609107.3   
ΒΟΡΕΙΟ ΑΙΓΑΙΟ                  1  0.2        184674.1    83103.3   184674.1   
ΔΥΤΙΚΗ ΕΛΛΑΔΑ                 10  2.1       1274856.9   568526.0  1270222.1   
ΔΥΤΙΚΗ ΜΑΚΕΔΟΝΙΑ              22  4.5       2876760.5  1281602.3  2806263.5   
ΗΠΕΙΡΟΣ                        7  1.4        539119.0   259177.8   537954.0   
ΘΕΣΣΑΛΙΑ                       6  1.2        642585.9   287679.5   639287.8   
ΚΕΝΤΡΙΚΗ ΜΑΚΕΔΟΝΙΑ           132 27.2      16160703.2  7173945.8 15879039.6   
ΚΡΗΤΗ                         16  3.3       1954718.2   889924.5  1935921.5   
ΝΟΤΙΟ ΑΙΓΑΙΟ                   1  0.2         57740.0    25983.0    57740.0   
ΠΕΛΟΠΟΝΝΗΣΟΣ                  10  2.1       1358611.7   615874.0  1355283.1   
ΣΤΕΡΕΑ ΕΛΛΑΔΑ                 22  4.5       3329927.6  1480518.1  3242094.0   

                            Κατανομή κόστους(%)           Mέσος όρος (€x10^6)  \
                                             ΣΠ   ΕΠ  ΔΔΠ                  ΣΠ   
Περιφέρεια                                                                      
ΑΝΑΤΟΛΙΚΗ ΜΑΚΕΔΟΝΙΑ & ΘΡΑΚΗ                 2.9  2.8  2.8            105480.1   
ΑΤΤΙΚΗ                                     45.8 45.6 45.5            103731.6   
ΒΟΡΕΙΟ ΑΙΓΑΙΟ                               0.3  0.3  0.3            184674.1   
ΔΥΤΙΚΗ ΕΛΛΑΔΑ                               2.3  2.3  2.3            127485.7   
ΔΥΤΙΚΗ ΜΑΚΕΔΟΝΙΑ                            5.2  5.2  5.2            130761.8   
ΗΠΕΙΡΟΣ                                     1.0  1.1  1.0             77017.0   
ΘΕΣΣΑΛΙΑ                                    1.2  1.2  1.2            107097.7   
ΚΕΝΤΡΙΚΗ ΜΑΚΕΔΟΝΙΑ                         29.2 29.2 29.4            122429.6   
ΚΡΗΤΗ                                       3.5  3.6  3.6            122169.9   
ΝΟΤΙΟ ΑΙΓΑΙΟ                                0.1  0.1  0.1             57740.0   
ΠΕΛΟΠΟΝΝΗΣΟΣ                                2.5  2.5  2.5            135861.2   
ΣΤΕΡΕΑ ΕΛΛΑΔΑ                               6.0  6.0  6.0            151360.3   

                                              
                                 ΕΠ      ΔΔΠ  
Περιφέρεια                                    
ΑΝΑΤΟΛΙΚΗ ΜΑΚΕΔΟΝΙΑ & ΘΡΑΚΗ 46579.1 102314.9  
ΑΤΤΙΚΗ                      45844.7 100857.0  
ΒΟΡΕΙΟ ΑΙΓΑΙΟ               83103.3 184674.1  
ΔΥΤΙΚΗ ΕΛΛΑΔΑ               56852.6 127022.2  
ΔΥΤΙΚΗ ΜΑΚΕΔΟΝΙΑ            58254.7 127557.4  
ΗΠΕΙΡΟΣ                     37025.4  76850.6  
ΘΕΣΣΑΛΙΑ                    47946.6 106548.0  
ΚΕΝΤΡΙΚΗ ΜΑΚΕΔΟΝΙΑ          54348.1 120295.8  
ΚΡΗΤΗ                       55620.3 120995.1  
ΝΟΤΙΟ ΑΙΓΑΙΟ                25983.0  57740.0  
ΠΕΛΟΠΟΝΝΗΣΟΣ                61587.4 135528.3  
ΣΤΕΡΕΑ ΕΛΛΑΔΑ               67296.3 147367.9

### Format B

This is the format you asked for. Easier to implement and no legend is required, but less readable IMHO

In [32]:
RENAMES2 = {
    "projects": "Έργα",
    "budget": "Συνολικός προϋπολογισμός",
    "subsidized": "Επιχορηγούμενος προϋπολογισμός",
    "granted": "Δημόσια δαπάνη ολοκλήρωσης",
    "absolute": "Kόστος (€x10^6)",
    "relative": "(%)",
    "average": "M.O. (€x10^6)",
    "N": "Ν",
    "p": "(%)",
    "size": "Μέγεθος",
    "sector": "Τομέας",
    "prefecture": "Περιφέρεια",
}

def create_grouped_table2(df, by, rename=True):
    table = pd.concat(
        [
            get_no_projects(df, by),
            get_grouped_data(df, 'budget', by),
            get_grouped_data(df, 'subsidized', by),
            get_grouped_data(df, 'granted', by),
        ],
        axis=1,
        keys=['projects', 'budget', 'subsidized', 'granted']
    )
    if rename:
        table = table.reset_index().rename(columns=RENAMES2).set_index(RENAMES2[by])
    return table

create_grouped_table2(df, "size")
create_grouped_table2(df, "sector")
create_grouped_table2(df, "prefecture")

Έργα      Συνολικός προϋπολογισμός                     \
                          Ν  (%)          Kόστος (€x10^6)  (%) M.O. (€x10^6)   
Μέγεθος                                                                        
Μεγάλες                   7  1.4                1343610.9  2.4      191944.4   
Μεσαίες                 117 24.1               16394101.5 29.7      140120.5   
Μικρές και Πολύ Μικρές  362 74.5               37534707.8 67.9      103687.0   

                       Επιχορηγούμενος προϋπολογισμός                     \
                                      Kόστος (€x10^6)  (%) M.O. (€x10^6)   
Μέγεθος                                                                    
Μεγάλες                                     1332696.5  2.5      190385.2   
Μεσαίες                                    16075613.0 29.7      137398.4   
Μικρές και Πολύ Μικρές                     36644001.4 67.8      101226.5   

                       Δημόσια δαπάνη ολοκλήρωσης                     
                                  Kόστος (€x10^6)  (%) M.O. (€x10^6)  
Μέγεθος                                                               
Μεγάλες                                  532853.9  2.2       76122.0  
Μεσαίες                                 7286229.2 29.7       62275.5  
Μικρές και Πολύ Μικρές                 16732041.8 68.2       46221.1

Έργα      Συνολικός προϋπολογισμός                     \
                   Ν  (%)          Kόστος (€x10^6)  (%) M.O. (€x10^6)   
Τομέας                                                                  
Εμπόριο           23  4.7                3804814.5  6.9      165426.7   
Επαγ/Επιστ. Δρ.   31  6.4                2130415.1  3.9       68723.1   
Κατασκευές        13  2.7                 925794.4  1.7       71215.0   
Μεταποίηση       315 64.8               37503875.9 67.9      119059.9   
ΤΠΕ              102 21.0               10693094.3 19.3      104834.3   
Υγεία              2  0.4                 214426.0  0.4      107213.0   

                Επιχορηγούμενος προϋπολογισμός                     \
                               Kόστος (€x10^6)  (%) M.O. (€x10^6)   
Τομέας                                                              
Εμπόριο                              3770496.4  7.0      163934.6   
Επαγ/Επιστ. Δρ.                      2088316.7  3.9       67365.1   
Κατασκευές                            917316.7  1.7       70562.8   
Μεταποίηση                          36701685.0 67.9      116513.3   
ΤΠΕ                                 10360070.2 19.2      101569.3   
Υγεία                                 214426.0  0.4      107213.0   

                Δημόσια δαπάνη ολοκλήρωσης                     
                           Kόστος (€x10^6)  (%) M.O. (€x10^6)  
Τομέας                                                         
Εμπόριο                          1861148.9  7.6       80919.5  
Επαγ/Επιστ. Δρ.                   955797.9  3.9       30832.2  
Κατασκευές                        404262.1  1.6       31097.1  
Μεταποίηση                      16555866.1 67.4       52558.3  
ΤΠΕ                              4677934.5 19.1       45862.1  
Υγεία                              96115.4  0.4       48057.7

Έργα      Συνολικός προϋπολογισμός       \
                               Ν  (%)          Kόστος (€x10^6)  (%)   
Περιφέρεια                                                            
ΑΝΑΤΟΛΙΚΗ ΜΑΚΕΔΟΝΙΑ & ΘΡΑΚΗ   15  3.1                1582200.9  2.9   
ΑΤΤΙΚΗ                       244 50.2               25310522.2 45.8   
ΒΟΡΕΙΟ ΑΙΓΑΙΟ                  1  0.2                 184674.1  0.3   
ΔΥΤΙΚΗ ΕΛΛΑΔΑ                 10  2.1                1274856.9  2.3   
ΔΥΤΙΚΗ ΜΑΚΕΔΟΝΙΑ              22  4.5                2876760.5  5.2   
ΗΠΕΙΡΟΣ                        7  1.4                 539119.0  1.0   
ΘΕΣΣΑΛΙΑ                       6  1.2                 642585.9  1.2   
ΚΕΝΤΡΙΚΗ ΜΑΚΕΔΟΝΙΑ           132 27.2               16160703.2 29.2   
ΚΡΗΤΗ                         16  3.3                1954718.2  3.5   
ΝΟΤΙΟ ΑΙΓΑΙΟ                   1  0.2                  57740.0  0.1   
ΠΕΛΟΠΟΝΝΗΣΟΣ                  10  2.1                1358611.7  2.5   
ΣΤΕΡΕΑ ΕΛΛΑΔΑ                 22  4.5                3329927.6  6.0   

                                          Επιχορηγούμενος προϋπολογισμός       \
                            M.O. (€x10^6)                Kόστος (€x10^6)  (%)   
Περιφέρεια                                                                      
ΑΝΑΤΟΛΙΚΗ ΜΑΚΕΔΟΝΙΑ & ΘΡΑΚΗ      105480.1                      1534724.0  2.8   
ΑΤΤΙΚΗ                           103731.6                     24609107.3 45.5   
ΒΟΡΕΙΟ ΑΙΓΑΙΟ                    184674.1                       184674.1  0.3   
ΔΥΤΙΚΗ ΕΛΛΑΔΑ                    127485.7                      1270222.1  2.3   
ΔΥΤΙΚΗ ΜΑΚΕΔΟΝΙΑ                 130761.8                      2806263.5  5.2   
ΗΠΕΙΡΟΣ                           77017.0                       537954.0  1.0   
ΘΕΣΣΑΛΙΑ                         107097.7                       639287.8  1.2   
ΚΕΝΤΡΙΚΗ ΜΑΚΕΔΟΝΙΑ               122429.6                     15879039.6 29.4   
ΚΡΗΤΗ                            122169.9                      1935921.5  3.6   
ΝΟΤΙΟ ΑΙΓΑΙΟ                      57740.0                        57740.0  0.1   
ΠΕΛΟΠΟΝΝΗΣΟΣ                     135861.2                      1355283.1  2.5   
ΣΤΕΡΕΑ ΕΛΛΑΔΑ                    151360.3                      3242094.0  6.0   

                                          Δημόσια δαπάνη ολοκλήρωσης       \
                            M.O. (€x10^6)            Kόστος (€x10^6)  (%)   
Περιφέρεια                                                                  
ΑΝΑΤΟΛΙΚΗ ΜΑΚΕΔΟΝΙΑ & ΘΡΑΚΗ      102314.9                   698685.9  2.8   
ΑΤΤΙΚΗ                           100857.0                 11186104.6 45.6   
ΒΟΡΕΙΟ ΑΙΓΑΙΟ                    184674.1                    83103.3  0.3   
ΔΥΤΙΚΗ ΕΛΛΑΔΑ                    127022.2                   568526.0  2.3   
ΔΥΤΙΚΗ ΜΑΚΕΔΟΝΙΑ                 127557.4                  1281602.3  5.2   
ΗΠΕΙΡΟΣ                           76850.6                   259177.8  1.1   
ΘΕΣΣΑΛΙΑ                         106548.0                   287679.5  1.2   
ΚΕΝΤΡΙΚΗ ΜΑΚΕΔΟΝΙΑ               120295.8                  7173945.8 29.2   
ΚΡΗΤΗ                            120995.1                   889924.5  3.6   
ΝΟΤΙΟ ΑΙΓΑΙΟ                      57740.0                    25983.0  0.1   
ΠΕΛΟΠΟΝΝΗΣΟΣ                     135528.3                   615874.0  2.5   
ΣΤΕΡΕΑ ΕΛΛΑΔΑ                    147367.9                  1480518.1  6.0   

                                           
                            M.O. (€x10^6)  
Περιφέρεια                                 
ΑΝΑΤΟΛΙΚΗ ΜΑΚΕΔΟΝΙΑ & ΘΡΑΚΗ       46579.1  
ΑΤΤΙΚΗ                            45844.7  
ΒΟΡΕΙΟ ΑΙΓΑΙΟ                     83103.3  
ΔΥΤΙΚΗ ΕΛΛΑΔΑ                     56852.6  
ΔΥΤΙΚΗ ΜΑΚΕΔΟΝΙΑ                  58254.7  
ΗΠΕΙΡΟΣ                           37025.4  
ΘΕΣΣΑΛΙΑ                          47946.6  
ΚΕΝΤΡΙΚΗ ΜΑΚΕΔΟΝΙΑ                54348.1  
ΚΡΗΤΗ                             55620.3  
ΝΟΤΙΟ ΑΙΓΑΙΟ                      25983.0  
ΠΕΛΟΠΟΝΝΗΣΟΣ      